In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [41]:
df= pd.read_csv("Data.csv")
df["Country"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [42]:
#We need first three columns as independent variables , most of the times the dependent variable is situated at the end

X = df.iloc[: , :-1].values # Here we selected all the rows , as for the colums we selected all columns except the last one (-1)

#Purchased will the column reperesenting dependent variable 
Y = df.iloc[: , -1].values




The code contains few missing data and it needs to be handled

In [43]:
from sklearn.impute import SimpleImputer
Imp = SimpleImputer(missing_values = np.nan , strategy ="mean")
Imp.fit(X[: , 1:])
X[: , 1:] = Imp.transform(X[: , 1:])
X


array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, 63777.77777777778],
       ['France', 35.0, 58000.0],
       ['Spain', 38.77777777777778, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)

One-hot Encoding 

In [44]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0])], remainder = 'passthrough')
#ColumnTransformer takes a tuple defining the type of tranformation , the exact transfromation and columns to be affected
#remainder = 'passthrough' allows us to keep other unaffected columns from the dataframe in the output

In [45]:
X = ct.fit_transform(X) 
X

array([[1.0, 0.0, 0.0, 44.0, 72000.0],
       [0.0, 0.0, 1.0, 27.0, 48000.0],
       [0.0, 1.0, 0.0, 30.0, 54000.0],
       [0.0, 0.0, 1.0, 38.0, 61000.0],
       [0.0, 1.0, 0.0, 40.0, 63777.77777777778],
       [1.0, 0.0, 0.0, 35.0, 58000.0],
       [0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
       [1.0, 0.0, 0.0, 48.0, 79000.0],
       [0.0, 1.0, 0.0, 50.0, 83000.0],
       [1.0, 0.0, 0.0, 37.0, 67000.0]], dtype=object)

Encoding Dependent variable

In [46]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(Y)
Y

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

Train test split

In [47]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test = train_test_split(X,Y, test_size = 0.2 ,random_state = 1) #random_state is just a seed

In [48]:
X_train

array([[0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
       [0.0, 1.0, 0.0, 40.0, 63777.77777777778],
       [1.0, 0.0, 0.0, 44.0, 72000.0],
       [0.0, 0.0, 1.0, 38.0, 61000.0],
       [0.0, 0.0, 1.0, 27.0, 48000.0],
       [1.0, 0.0, 0.0, 48.0, 79000.0],
       [0.0, 1.0, 0.0, 50.0, 83000.0],
       [1.0, 0.0, 0.0, 35.0, 58000.0]], dtype=object)

Scaling data 

It must be done after splitting , since it can cause data leakage if done earlier 

For the above features we cannot apply scaling on the columns created by one hot encoding because its already between 0 and 1 , and that numerical data corresponds to a country which needs to be stay as it is

In [49]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[: , 3:] = scaler.fit_transform(X_train[: , 3:]) # scaler fit method returns the mean and SD of the data
#Here we will use the same scaler instance on test data with the mean and SD of training set
#With this technique we can tranform the test data into same format and get some relevant predictions
X_test[:, 3:] = scaler.transform(X_test[:, 3:])

In [50]:
X_train

array([[0.0, 0.0, 1.0, -0.19159184384578545, -1.0781259408412425],
       [0.0, 1.0, 0.0, -0.014117293757057777, -0.07013167641635372],
       [1.0, 0.0, 0.0, 0.566708506533324, 0.633562432710455],
       [0.0, 0.0, 1.0, -0.30453019390224867, -0.30786617274297867],
       [0.0, 0.0, 1.0, -1.9018011447007988, -1.420463615551582],
       [1.0, 0.0, 0.0, 1.1475343068237058, 1.232653363453549],
       [0.0, 1.0, 0.0, 1.4379472069688968, 1.5749910381638885],
       [1.0, 0.0, 0.0, -0.7401495441200351, -0.5646194287757332]],
      dtype=object)

Training and predicting